In [14]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score

In [9]:
train_data = pd.read_csv('train/train_all.csv')
test_data = pd.read_csv('test/republish_test.csv')

In [18]:
def filln(data):
    data = data.replace('\N', -1)
    data.gender = data.gender.astype(int)
    data.age = data.age.astype(int)
    data['3_total_fee'] = data['3_total_fee'].astype(float)
    data['2_total_fee'] = data['2_total_fee'].astype(float)
    return data
    
def trans1(data):
    data.current_service = data.current_service.replace(90063345, 0)
    data.current_service = data.current_service.replace(90109916, 1)
    data.current_service = data.current_service.replace(90155946, 2)
    return data

def trans4(data):
    data.current_service = data.current_service.replace(89950166, 0)
    data.current_service = data.current_service.replace(89950167, 1)
    data.current_service = data.current_service.replace(99999828, 2)
    data.current_service = data.current_service.replace(89950168, 3)
    data.current_service = data.current_service.replace(99999827, 4)
    data.current_service = data.current_service.replace(99999826, 5)
    data.current_service = data.current_service.replace(99999830, 6)
    data.current_service = data.current_service.replace(99999825, 7)
    return data



In [48]:
train_data = filln(train_data)
test_data = filln(test_data)
test_data.service_type.describe()

count    200000.000000
mean          2.007635
std           1.416826
min           1.000000
25%           1.000000
50%           1.000000
75%           4.000000
max           4.000000
Name: service_type, dtype: float64

In [30]:
train_data1 = train_data.loc[train_data.service_type == 1]
# train_data1 = train_data1.loc[train_data1.is_promise_low_consume == 1]

train_data4 = train_data.loc[train_data.service_type == 4]

test_data1 = test_data.loc[test_data.service_type == 1]
# test_data1_1 = test_data1.loc[test_data1.is_promise_low_consume == 0]

# test_data1 = test_data1.loc[test_data.is_promise_low_consume == 1]

test_data3 = test_data.loc[test_data.service_type == 3]

test_data4 = test_data.loc[test_data.service_type == 4]

In [28]:
# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

train_data1 = trans1(train_data1)

feature1 = [u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',]

# 'service_type', u'is_mix_service', u'is_promise_low_consume', 

train_x, evals_x, train_y, evals_y = train_test_split(train_data1[feature1], train_data1.current_service,
                                                      test_size=0.2, random_state=2018)

print train_x.shape
print("LGB test")
params={
    "learning_rate":0.1,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":4,
    "objective":"multiclass",
    "num_class":3,
    "silent":True,
}
t_data = lgb.Dataset(train_x, label=train_y)
v_data = lgb.Dataset(evals_x, label=evals_y)
clf1=lgb.train(params,t_data,num_boost_round=100000,valid_sets=[t_data,v_data],
              early_stopping_rounds=50,feval=f1_score_vali,verbose_eval=1)

(277881, 22)
LGB test
[1]	training's multi_logloss: 0.965211	training's f1_score: 0.978754	valid_1's multi_logloss: 0.965287	valid_1's f1_score: 0.978098
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_logloss: 0.85455	training's f1_score: 0.978112	valid_1's multi_logloss: 0.85463	valid_1's f1_score: 0.977385
[3]	training's multi_logloss: 0.760419	training's f1_score: 0.978433	valid_1's multi_logloss: 0.760543	valid_1's f1_score: 0.97777
[4]	training's multi_logloss: 0.680216	training's f1_score: 0.978421	valid_1's multi_logloss: 0.680337	valid_1's f1_score: 0.977745
[5]	training's multi_logloss: 0.610603	training's f1_score: 0.978577	valid_1's multi_logloss: 0.61076	valid_1's f1_score: 0.977962
[6]	training's multi_logloss: 0.550324	training's f1_score: 0.978651	valid_1's multi_logloss: 0.550475	valid_1's f1_score: 0.978008
[7]	training's multi_logloss: 0.497211	training's f1_score: 0.979087	valid_1's multi_logloss: 0.497382	valid_1's f1_score: 0.978

[63]	training's multi_logloss: 0.0253064	training's f1_score: 0.99248	valid_1's multi_logloss: 0.0268305	valid_1's f1_score: 0.991421
[64]	training's multi_logloss: 0.0248654	training's f1_score: 0.992539	valid_1's multi_logloss: 0.026405	valid_1's f1_score: 0.991552
[65]	training's multi_logloss: 0.0244795	training's f1_score: 0.992633	valid_1's multi_logloss: 0.0260484	valid_1's f1_score: 0.991595
[66]	training's multi_logloss: 0.0240775	training's f1_score: 0.992687	valid_1's multi_logloss: 0.025678	valid_1's f1_score: 0.99177
[67]	training's multi_logloss: 0.0237422	training's f1_score: 0.992738	valid_1's multi_logloss: 0.0253629	valid_1's f1_score: 0.99177
[68]	training's multi_logloss: 0.0233957	training's f1_score: 0.992802	valid_1's multi_logloss: 0.0250398	valid_1's f1_score: 0.991901
[69]	training's multi_logloss: 0.0230584	training's f1_score: 0.992889	valid_1's multi_logloss: 0.0247224	valid_1's f1_score: 0.991987
[70]	training's multi_logloss: 0.022739	training's f1_score:

[125]	training's multi_logloss: 0.0145356	training's f1_score: 0.995086	valid_1's multi_logloss: 0.0169898	valid_1's f1_score: 0.994106
[126]	training's multi_logloss: 0.0144796	training's f1_score: 0.995104	valid_1's multi_logloss: 0.0169391	valid_1's f1_score: 0.994163
[127]	training's multi_logloss: 0.0144181	training's f1_score: 0.995114	valid_1's multi_logloss: 0.016888	valid_1's f1_score: 0.994135
[128]	training's multi_logloss: 0.0143625	training's f1_score: 0.995118	valid_1's multi_logloss: 0.0168352	valid_1's f1_score: 0.994163
[129]	training's multi_logloss: 0.0142727	training's f1_score: 0.995137	valid_1's multi_logloss: 0.016755	valid_1's f1_score: 0.994222
[130]	training's multi_logloss: 0.0142162	training's f1_score: 0.995151	valid_1's multi_logloss: 0.0167134	valid_1's f1_score: 0.994236
[131]	training's multi_logloss: 0.0141657	training's f1_score: 0.995154	valid_1's multi_logloss: 0.016679	valid_1's f1_score: 0.994193
[132]	training's multi_logloss: 0.0141173	training'

[186]	training's multi_logloss: 0.0116068	training's f1_score: 0.99598	valid_1's multi_logloss: 0.0147949	valid_1's f1_score: 0.994739
[187]	training's multi_logloss: 0.0115795	training's f1_score: 0.995991	valid_1's multi_logloss: 0.0147762	valid_1's f1_score: 0.994754
[188]	training's multi_logloss: 0.0115431	training's f1_score: 0.995995	valid_1's multi_logloss: 0.0147524	valid_1's f1_score: 0.994798
[189]	training's multi_logloss: 0.0115165	training's f1_score: 0.995991	valid_1's multi_logloss: 0.0147354	valid_1's f1_score: 0.994812
[190]	training's multi_logloss: 0.011482	training's f1_score: 0.996009	valid_1's multi_logloss: 0.0147227	valid_1's f1_score: 0.994798
[191]	training's multi_logloss: 0.0114423	training's f1_score: 0.99602	valid_1's multi_logloss: 0.0146924	valid_1's f1_score: 0.994797
[192]	training's multi_logloss: 0.0113921	training's f1_score: 0.996042	valid_1's multi_logloss: 0.0146545	valid_1's f1_score: 0.994869
[193]	training's multi_logloss: 0.011365	training's

[248]	training's multi_logloss: 0.00983468	training's f1_score: 0.996685	valid_1's multi_logloss: 0.0137879	valid_1's f1_score: 0.995128
[249]	training's multi_logloss: 0.00981548	training's f1_score: 0.996689	valid_1's multi_logloss: 0.0137757	valid_1's f1_score: 0.995143
[250]	training's multi_logloss: 0.00979499	training's f1_score: 0.996704	valid_1's multi_logloss: 0.0137624	valid_1's f1_score: 0.995114
[251]	training's multi_logloss: 0.0097725	training's f1_score: 0.996707	valid_1's multi_logloss: 0.0137504	valid_1's f1_score: 0.9951
[252]	training's multi_logloss: 0.00975378	training's f1_score: 0.996711	valid_1's multi_logloss: 0.013746	valid_1's f1_score: 0.995142
[253]	training's multi_logloss: 0.00973435	training's f1_score: 0.996725	valid_1's multi_logloss: 0.0137315	valid_1's f1_score: 0.995127
[254]	training's multi_logloss: 0.00970397	training's f1_score: 0.996721	valid_1's multi_logloss: 0.0137169	valid_1's f1_score: 0.99517
[255]	training's multi_logloss: 0.00968521	tra

[308]	training's multi_logloss: 0.00869966	training's f1_score: 0.997135	valid_1's multi_logloss: 0.0132271	valid_1's f1_score: 0.995299
[309]	training's multi_logloss: 0.00868054	training's f1_score: 0.997138	valid_1's multi_logloss: 0.0132277	valid_1's f1_score: 0.995299
[310]	training's multi_logloss: 0.00865998	training's f1_score: 0.997146	valid_1's multi_logloss: 0.0132234	valid_1's f1_score: 0.995299
[311]	training's multi_logloss: 0.00864323	training's f1_score: 0.99716	valid_1's multi_logloss: 0.0132183	valid_1's f1_score: 0.995285
[312]	training's multi_logloss: 0.00862858	training's f1_score: 0.997167	valid_1's multi_logloss: 0.0132207	valid_1's f1_score: 0.995285
[313]	training's multi_logloss: 0.00861633	training's f1_score: 0.997164	valid_1's multi_logloss: 0.0132211	valid_1's f1_score: 0.995285
[314]	training's multi_logloss: 0.0086032	training's f1_score: 0.997174	valid_1's multi_logloss: 0.013215	valid_1's f1_score: 0.995328
[315]	training's multi_logloss: 0.00858401	t

[368]	training's multi_logloss: 0.00780165	training's f1_score: 0.997469	valid_1's multi_logloss: 0.0129895	valid_1's f1_score: 0.995327
[369]	training's multi_logloss: 0.00778916	training's f1_score: 0.997476	valid_1's multi_logloss: 0.0129824	valid_1's f1_score: 0.995313
[370]	training's multi_logloss: 0.00777505	training's f1_score: 0.997498	valid_1's multi_logloss: 0.0129777	valid_1's f1_score: 0.995342
[371]	training's multi_logloss: 0.0077591	training's f1_score: 0.997502	valid_1's multi_logloss: 0.0129761	valid_1's f1_score: 0.995371
[372]	training's multi_logloss: 0.00774144	training's f1_score: 0.997509	valid_1's multi_logloss: 0.0129655	valid_1's f1_score: 0.995386
[373]	training's multi_logloss: 0.00772533	training's f1_score: 0.997498	valid_1's multi_logloss: 0.0129672	valid_1's f1_score: 0.995357
[374]	training's multi_logloss: 0.00771027	training's f1_score: 0.997509	valid_1's multi_logloss: 0.0129637	valid_1's f1_score: 0.995371
[375]	training's multi_logloss: 0.00769691

[428]	training's multi_logloss: 0.00702982	training's f1_score: 0.997815	valid_1's multi_logloss: 0.012818	valid_1's f1_score: 0.995371
[429]	training's multi_logloss: 0.00702207	training's f1_score: 0.997815	valid_1's multi_logloss: 0.0128156	valid_1's f1_score: 0.995371
[430]	training's multi_logloss: 0.00701357	training's f1_score: 0.997825	valid_1's multi_logloss: 0.0128166	valid_1's f1_score: 0.995357
[431]	training's multi_logloss: 0.00699443	training's f1_score: 0.997815	valid_1's multi_logloss: 0.0128066	valid_1's f1_score: 0.995386
[432]	training's multi_logloss: 0.00698405	training's f1_score: 0.997829	valid_1's multi_logloss: 0.0128071	valid_1's f1_score: 0.995357
[433]	training's multi_logloss: 0.00697216	training's f1_score: 0.997836	valid_1's multi_logloss: 0.0128017	valid_1's f1_score: 0.995357
[434]	training's multi_logloss: 0.00696685	training's f1_score: 0.99784	valid_1's multi_logloss: 0.0127983	valid_1's f1_score: 0.995357
[435]	training's multi_logloss: 0.00695668	

In [36]:
pre = clf1.predict(test_data1[feature1].values,num_iteration=clf1.best_iteration)
test_data1['predict'] = [np.argmax(x) for x in pre]
test_data1['predict']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0         0
1         0
2         0
3         0
4         0
5         0
9         0
11        0
13        1
14        0
15        0
18        0
19        0
21        0
22        0
24        0
25        0
26        0
28        1
29        0
30        0
31        0
32        0
36        0
40        0
43        0
46        0
47        0
49        0
50        0
         ..
199952    1
199953    0
199954    0
199955    0
199956    0
199957    0
199958    0
199961    0
199965    0
199968    0
199969    0
199970    0
199972    0
199973    0
199975    0
199977    2
199978    0
199979    0
199981    2
199985    0
199986    0
199987    0
199989    0
199990    0
199992    0
199993    1
199995    0
199996    0
199997    0
199998    0
Name: predict, Length: 132818, dtype: int64

In [42]:
train_data1 = trans1(train_data1)

feature1 = [u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',]

# 'service_type', u'is_mix_service', u'is_promise_low_consume', 

train_x, evals_x, train_y, evals_y = train_test_split(train_data1[feature1], train_data1.current_service,
                                                      test_size=0.2, random_state=2018)

print train_x.shape
print("LGB test")
clf1 = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=8, reg_alpha=0.0, reg_lambda=1,
    max_depth=3, n_estimators=10000, objective='multi-class',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.2, min_child_weight=50, random_state=2018, n_jobs=-1,
    )
clf1.fit(train_x, train_y, eval_set=[(train_x, train_y),(evals_x, evals_y)], 
        eval_metric='multi_error',early_stopping_rounds=50, 
        categorical_feature=['many_over_bill', 'gender', 'contract_type'],
        )

# callbacks = f1_score_vali

(277881, 22)
LGB test
[1]	training's multi_error: 0.031733	valid_1's multi_error: 0.0321717
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_error: 0.0275262	valid_1's multi_error: 0.027767
[3]	training's multi_error: 0.0255289	valid_1's multi_error: 0.0254063
[4]	training's multi_error: 0.0239527	valid_1's multi_error: 0.0238085
[5]	training's multi_error: 0.025187	valid_1's multi_error: 0.0254783
[6]	training's multi_error: 0.0250575	valid_1's multi_error: 0.025104
[7]	training's multi_error: 0.0226824	valid_1's multi_error: 0.0226857
[8]	training's multi_error: 0.021646	valid_1's multi_error: 0.0218364
[9]	training's multi_error: 0.0214912	valid_1's multi_error: 0.0219804
[10]	training's multi_error: 0.0196523	valid_1's multi_error: 0.0201523
[11]	training's multi_error: 0.0200194	valid_1's multi_error: 0.0204114
[12]	training's multi_error: 0.0193068	valid_1's multi_error: 0.0197205
[13]	training's multi_error: 0.0191737	valid_1's multi_error: 0.01

[115]	training's multi_error: 0.00539799	valid_1's multi_error: 0.00588735
[116]	training's multi_error: 0.00539799	valid_1's multi_error: 0.00590174
[117]	training's multi_error: 0.0053944	valid_1's multi_error: 0.00594493
[118]	training's multi_error: 0.00536561	valid_1's multi_error: 0.00590174
[119]	training's multi_error: 0.00534761	valid_1's multi_error: 0.00582977
[120]	training's multi_error: 0.00536561	valid_1's multi_error: 0.00584417
[121]	training's multi_error: 0.00534761	valid_1's multi_error: 0.00594493
[122]	training's multi_error: 0.00532242	valid_1's multi_error: 0.00588735
[123]	training's multi_error: 0.00530803	valid_1's multi_error: 0.00585856
[124]	training's multi_error: 0.00534042	valid_1's multi_error: 0.00584417
[125]	training's multi_error: 0.00527924	valid_1's multi_error: 0.00580098
[126]	training's multi_error: 0.00524685	valid_1's multi_error: 0.00578659
[127]	training's multi_error: 0.00523965	valid_1's multi_error: 0.00580098
[128]	training's multi_err

[225]	training's multi_error: 0.00428961	valid_1's multi_error: 0.00526839
[226]	training's multi_error: 0.0042968	valid_1's multi_error: 0.00525399
[227]	training's multi_error: 0.0042932	valid_1's multi_error: 0.00526839
[228]	training's multi_error: 0.0042932	valid_1's multi_error: 0.00526839
[229]	training's multi_error: 0.00428961	valid_1's multi_error: 0.00525399
[230]	training's multi_error: 0.0042968	valid_1's multi_error: 0.00529717
[231]	training's multi_error: 0.0043148	valid_1's multi_error: 0.00532596
[232]	training's multi_error: 0.0042932	valid_1's multi_error: 0.00528278
[233]	training's multi_error: 0.0043004	valid_1's multi_error: 0.00525399
[234]	training's multi_error: 0.0043076	valid_1's multi_error: 0.00528278
[235]	training's multi_error: 0.0043076	valid_1's multi_error: 0.00535475
[236]	training's multi_error: 0.00428961	valid_1's multi_error: 0.00532596
[237]	training's multi_error: 0.0043004	valid_1's multi_error: 0.00532596
[238]	training's multi_error: 0.004

[337]	training's multi_error: 0.00379659	valid_1's multi_error: 0.00486534
[338]	training's multi_error: 0.00380739	valid_1's multi_error: 0.00492292
[339]	training's multi_error: 0.00381098	valid_1's multi_error: 0.00487973
[340]	training's multi_error: 0.00381458	valid_1's multi_error: 0.00487973
[341]	training's multi_error: 0.00380019	valid_1's multi_error: 0.00485094
[342]	training's multi_error: 0.00379299	valid_1's multi_error: 0.00489413
[343]	training's multi_error: 0.00378219	valid_1's multi_error: 0.00487973
[344]	training's multi_error: 0.00378939	valid_1's multi_error: 0.00489413
[345]	training's multi_error: 0.00380019	valid_1's multi_error: 0.00486534
[346]	training's multi_error: 0.00379659	valid_1's multi_error: 0.00490852
[347]	training's multi_error: 0.00381458	valid_1's multi_error: 0.00492292
[348]	training's multi_error: 0.00380019	valid_1's multi_error: 0.00489413
[349]	training's multi_error: 0.00380379	valid_1's multi_error: 0.00490852
[350]	training's multi_er

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        learning_rate=0.2, max_depth=3, min_child_samples=20,
        min_child_weight=50, min_split_gain=0.0, n_estimators=10000,
        n_jobs=-1, num_leaves=8, objective='multi-class',
        random_state=2018, reg_alpha=0.0, reg_lambda=1, silent=True,
        subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [23]:
test_data1['predict'] = clf1.predict(test_data1[feature1])

ValueError: Wrong number of items passed 3, placement implies 1

In [37]:
res1 = test_data1.loc[:,['user_id','predict']]
res1.describe()

,predict
count,132818.000000
mean,0.178748
std,0.500956
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,2.000000


In [38]:
res1.predict = res1.predict.replace(0, 90063345)
res1.predict = res1.predict.replace(1, 90109916)
res1.predict = res1.predict.replace(2, 90155946)
res1

,user_id,predict
0,012pSke7DsCrR985,90063345
1,012re3ZJSB6FptHW,90063345
2,013ElTBrRntuZDXz,90063345
3,0146B9qQbNxIynoX,90063345
4,0149vWf6u8UrKyJT,90063345
5,014fpazsyUghObB3,90063345
9,015sZpdVXKiB96a3,90063345
11,016YErq3KiVTJg7y,90063345
13,017YgyTbs4HPCpwc,90109916
14,018sgOxXUwu6qDZC,90063345


In [22]:
# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(8, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

from sklearn.cross_validation import train_test_split
import lightgbm as lgb
train_data4 = trans4(train_data4)

feature4 =[ u'is_mix_service', u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'is_promise_low_consume', u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',]

# u'service_type', 

train_x, evals_x, train_y, evals_y = train_test_split(train_data4[feature4], train_data4.current_service,
                                                      test_size=0.2, random_state=2018)

print train_x.shape
print("LGB test")
params={
    "learning_rate":0.1,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":4,
    "objective":"multiclass",
    "num_class":8,
    "silent":True,
}
t_data = lgb.Dataset(train_x, label=train_y)
v_data = lgb.Dataset(evals_x, label=evals_y)
clf2=lgb.train(params,t_data,num_boost_round=100000,valid_sets=[t_data,v_data],
              early_stopping_rounds=50,feval=f1_score_vali,verbose_eval=1)

(317310, 24)
LGB test
[1]	training's multi_logloss: 1.89484	training's f1_score: 0.68873	valid_1's multi_logloss: 1.89498	valid_1's f1_score: 0.68968
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_logloss: 1.75719	training's f1_score: 0.703717	valid_1's multi_logloss: 1.75724	valid_1's f1_score: 0.705595
[3]	training's multi_logloss: 1.6449	training's f1_score: 0.709336	valid_1's multi_logloss: 1.64501	valid_1's f1_score: 0.710609
[4]	training's multi_logloss: 1.54959	training's f1_score: 0.729368	valid_1's multi_logloss: 1.54965	valid_1's f1_score: 0.730146
[5]	training's multi_logloss: 1.46905	training's f1_score: 0.731967	valid_1's multi_logloss: 1.46921	valid_1's f1_score: 0.733043
[6]	training's multi_logloss: 1.39659	training's f1_score: 0.732294	valid_1's multi_logloss: 1.39687	valid_1's f1_score: 0.731976
[7]	training's multi_logloss: 1.3338	training's f1_score: 0.736115	valid_1's multi_logloss: 1.33402	valid_1's f1_score: 0.736518
[8]	traini

[63]	training's multi_logloss: 0.564439	training's f1_score: 0.808171	valid_1's multi_logloss: 0.566577	valid_1's f1_score: 0.808407
[64]	training's multi_logloss: 0.561322	training's f1_score: 0.808869	valid_1's multi_logloss: 0.563519	valid_1's f1_score: 0.809026
[65]	training's multi_logloss: 0.558276	training's f1_score: 0.809565	valid_1's multi_logloss: 0.560505	valid_1's f1_score: 0.809676
[66]	training's multi_logloss: 0.555491	training's f1_score: 0.810115	valid_1's multi_logloss: 0.557774	valid_1's f1_score: 0.810347
[67]	training's multi_logloss: 0.552395	training's f1_score: 0.811139	valid_1's multi_logloss: 0.554732	valid_1's f1_score: 0.811372
[68]	training's multi_logloss: 0.550011	training's f1_score: 0.811803	valid_1's multi_logloss: 0.552432	valid_1's f1_score: 0.811874
[69]	training's multi_logloss: 0.547251	training's f1_score: 0.812392	valid_1's multi_logloss: 0.549725	valid_1's f1_score: 0.812524
[70]	training's multi_logloss: 0.544733	training's f1_score: 0.81283	

[125]	training's multi_logloss: 0.461717	training's f1_score: 0.837387	valid_1's multi_logloss: 0.466369	valid_1's f1_score: 0.835615
[126]	training's multi_logloss: 0.460722	training's f1_score: 0.837729	valid_1's multi_logloss: 0.465399	valid_1's f1_score: 0.835501
[127]	training's multi_logloss: 0.459848	training's f1_score: 0.83799	valid_1's multi_logloss: 0.464534	valid_1's f1_score: 0.83585
[128]	training's multi_logloss: 0.458972	training's f1_score: 0.838306	valid_1's multi_logloss: 0.463702	valid_1's f1_score: 0.836394
[129]	training's multi_logloss: 0.457876	training's f1_score: 0.838607	valid_1's multi_logloss: 0.462654	valid_1's f1_score: 0.836809
[130]	training's multi_logloss: 0.457189	training's f1_score: 0.838849	valid_1's multi_logloss: 0.462019	valid_1's f1_score: 0.836923
[131]	training's multi_logloss: 0.456216	training's f1_score: 0.838985	valid_1's multi_logloss: 0.461122	valid_1's f1_score: 0.837039
[132]	training's multi_logloss: 0.455457	training's f1_score: 0.

[187]	training's multi_logloss: 0.419283	training's f1_score: 0.850741	valid_1's multi_logloss: 0.426511	valid_1's f1_score: 0.847619
[188]	training's multi_logloss: 0.418543	training's f1_score: 0.851084	valid_1's multi_logloss: 0.425825	valid_1's f1_score: 0.847714
[189]	training's multi_logloss: 0.418085	training's f1_score: 0.851241	valid_1's multi_logloss: 0.425429	valid_1's f1_score: 0.847914
[190]	training's multi_logloss: 0.417717	training's f1_score: 0.851301	valid_1's multi_logloss: 0.425112	valid_1's f1_score: 0.848059
[191]	training's multi_logloss: 0.417052	training's f1_score: 0.851615	valid_1's multi_logloss: 0.424496	valid_1's f1_score: 0.848003
[192]	training's multi_logloss: 0.416494	training's f1_score: 0.851649	valid_1's multi_logloss: 0.423993	valid_1's f1_score: 0.848217
[193]	training's multi_logloss: 0.415721	training's f1_score: 0.851956	valid_1's multi_logloss: 0.423251	valid_1's f1_score: 0.848385
[194]	training's multi_logloss: 0.415257	training's f1_score: 

[249]	training's multi_logloss: 0.393269	training's f1_score: 0.859193	valid_1's multi_logloss: 0.403048	valid_1's f1_score: 0.854511
[250]	training's multi_logloss: 0.392794	training's f1_score: 0.859287	valid_1's multi_logloss: 0.40261	valid_1's f1_score: 0.854629
[251]	training's multi_logloss: 0.392511	training's f1_score: 0.859427	valid_1's multi_logloss: 0.402403	valid_1's f1_score: 0.854577
[252]	training's multi_logloss: 0.392248	training's f1_score: 0.859543	valid_1's multi_logloss: 0.402191	valid_1's f1_score: 0.854849
[253]	training's multi_logloss: 0.391986	training's f1_score: 0.85952	valid_1's multi_logloss: 0.401976	valid_1's f1_score: 0.854942
[254]	training's multi_logloss: 0.391599	training's f1_score: 0.859699	valid_1's multi_logloss: 0.401624	valid_1's f1_score: 0.855039
[255]	training's multi_logloss: 0.391233	training's f1_score: 0.85977	valid_1's multi_logloss: 0.401321	valid_1's f1_score: 0.855048
[256]	training's multi_logloss: 0.390916	training's f1_score: 0.8

...

[1115]	training's multi_logloss: 0.279675	training's f1_score: 0.901412	valid_1's multi_logloss: 0.328237	valid_1's f1_score: 0.878884
[1116]	training's multi_logloss: 0.279623	training's f1_score: 0.901422	valid_1's multi_logloss: 0.328228	valid_1's f1_score: 0.878974
[1117]	training's multi_logloss: 0.279549	training's f1_score: 0.901462	valid_1's multi_logloss: 0.328188	valid_1's f1_score: 0.878951
[1118]	training's multi_logloss: 0.279465	training's f1_score: 0.901534	valid_1's multi_logloss: 0.328139	valid_1's f1_score: 0.879064
[1119]	training's multi_logloss: 0.2794	training's f1_score: 0.90157	valid_1's multi_logloss: 0.328118	valid_1's f1_score: 0.879053
[1120]	training's multi_logloss: 0.279325	training's f1_score: 0.90164	valid_1's multi_logloss: 0.328074	valid_1's f1_score: 0.879014
[1121]	training's multi_logloss: 0.279252	training's f1_score: 0.901698	valid_1's multi_logloss: 0.328031	valid_1's f1_score: 0.878911
[1122]	training's multi_logloss: 0.279173	training's f1_sco

[1176]	training's multi_logloss: 0.275558	training's f1_score: 0.903237	valid_1's multi_logloss: 0.326515	valid_1's f1_score: 0.879683
[1177]	training's multi_logloss: 0.275499	training's f1_score: 0.903295	valid_1's multi_logloss: 0.326484	valid_1's f1_score: 0.879695
[1178]	training's multi_logloss: 0.27542	training's f1_score: 0.90335	valid_1's multi_logloss: 0.326449	valid_1's f1_score: 0.87973
[1179]	training's multi_logloss: 0.275361	training's f1_score: 0.90335	valid_1's multi_logloss: 0.326426	valid_1's f1_score: 0.879742
[1180]	training's multi_logloss: 0.275297	training's f1_score: 0.903378	valid_1's multi_logloss: 0.326398	valid_1's f1_score: 0.879745
[1181]	training's multi_logloss: 0.275219	training's f1_score: 0.903464	valid_1's multi_logloss: 0.326359	valid_1's f1_score: 0.879845
[1182]	training's multi_logloss: 0.275149	training's f1_score: 0.903497	valid_1's multi_logloss: 0.326338	valid_1's f1_score: 0.87986
[1183]	training's multi_logloss: 0.275093	training's f1_scor

[1237]	training's multi_logloss: 0.27149	training's f1_score: 0.905181	valid_1's multi_logloss: 0.324859	valid_1's f1_score: 0.88075
[1238]	training's multi_logloss: 0.271425	training's f1_score: 0.90523	valid_1's multi_logloss: 0.324847	valid_1's f1_score: 0.880675
[1239]	training's multi_logloss: 0.271356	training's f1_score: 0.905288	valid_1's multi_logloss: 0.324818	valid_1's f1_score: 0.88071
[1240]	training's multi_logloss: 0.271298	training's f1_score: 0.905331	valid_1's multi_logloss: 0.324793	valid_1's f1_score: 0.88066
[1241]	training's multi_logloss: 0.271247	training's f1_score: 0.905334	valid_1's multi_logloss: 0.324774	valid_1's f1_score: 0.880713
[1242]	training's multi_logloss: 0.271174	training's f1_score: 0.905337	valid_1's multi_logloss: 0.324732	valid_1's f1_score: 0.880723
[1243]	training's multi_logloss: 0.271113	training's f1_score: 0.905337	valid_1's multi_logloss: 0.324706	valid_1's f1_score: 0.880785
[1244]	training's multi_logloss: 0.271054	training's f1_scor

[1298]	training's multi_logloss: 0.267552	training's f1_score: 0.906811	valid_1's multi_logloss: 0.323284	valid_1's f1_score: 0.881295
[1299]	training's multi_logloss: 0.267487	training's f1_score: 0.906824	valid_1's multi_logloss: 0.323258	valid_1's f1_score: 0.881268
[1300]	training's multi_logloss: 0.267437	training's f1_score: 0.906825	valid_1's multi_logloss: 0.323245	valid_1's f1_score: 0.881269
[1301]	training's multi_logloss: 0.267355	training's f1_score: 0.906885	valid_1's multi_logloss: 0.323203	valid_1's f1_score: 0.881306
[1302]	training's multi_logloss: 0.267292	training's f1_score: 0.90688	valid_1's multi_logloss: 0.323187	valid_1's f1_score: 0.881259
[1303]	training's multi_logloss: 0.267236	training's f1_score: 0.906934	valid_1's multi_logloss: 0.323171	valid_1's f1_score: 0.881257
[1304]	training's multi_logloss: 0.267176	training's f1_score: 0.90696	valid_1's multi_logloss: 0.323142	valid_1's f1_score: 0.88127
[1305]	training's multi_logloss: 0.267105	training's f1_sc

[1359]	training's multi_logloss: 0.263727	training's f1_score: 0.908403	valid_1's multi_logloss: 0.32184	valid_1's f1_score: 0.881713
[1360]	training's multi_logloss: 0.263682	training's f1_score: 0.908409	valid_1's multi_logloss: 0.321831	valid_1's f1_score: 0.881689
[1361]	training's multi_logloss: 0.263613	training's f1_score: 0.908454	valid_1's multi_logloss: 0.321795	valid_1's f1_score: 0.88175
[1362]	training's multi_logloss: 0.263536	training's f1_score: 0.908492	valid_1's multi_logloss: 0.321766	valid_1's f1_score: 0.881704
[1363]	training's multi_logloss: 0.26348	training's f1_score: 0.908503	valid_1's multi_logloss: 0.321748	valid_1's f1_score: 0.881691
[1364]	training's multi_logloss: 0.26342	training's f1_score: 0.908514	valid_1's multi_logloss: 0.321722	valid_1's f1_score: 0.881654
[1365]	training's multi_logloss: 0.263362	training's f1_score: 0.908535	valid_1's multi_logloss: 0.321701	valid_1's f1_score: 0.881613
[1366]	training's multi_logloss: 0.263302	training's f1_sco

In [39]:
pre = clf2.predict(test_data4[feature4].values,num_iteration=clf2.best_iteration)
test_data4['predict'] = [np.argmax(x) for x in pre]
test_data4['predict']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


6         4
7         1
8         0
10        6
12        2
16        0
17        2
20        0
23        4
27        2
33        1
34        5
35        5
37        3
38        1
39        1
41        5
42        7
44        0
45        1
48        4
59        5
60        2
62        5
66        1
69        1
70        1
71        2
74        6
75        0
         ..
199929    0
199930    1
199932    1
199935    1
199936    2
199938    1
199939    1
199940    2
199943    1
199945    5
199948    1
199949    1
199959    1
199960    0
199962    6
199963    0
199964    1
199966    1
199967    1
199971    0
199974    5
199976    1
199980    7
199982    5
199983    2
199984    0
199988    1
199991    0
199994    2
199999    3
Name: predict, Length: 67163, dtype: int64

In [43]:
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
train_data4 = trans4(train_data4)

feature4 =[ u'is_mix_service', u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'is_promise_low_consume', u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',]

# u'service_type', 

train_x, evals_x, train_y, evals_y = train_test_split(train_data4[feature4], train_data4.current_service,
                                                      test_size=0.2, random_state=2018)

print train_x.shape
print("LGB test")
clf2 = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=8, reg_alpha=0.0, reg_lambda=1,
    max_depth=3, n_estimators=1000, objective='multi-class', num_class = 8,
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=1, min_child_weight=50, random_state=2018, n_jobs=-1
)
clf2.fit(train_x, train_y, eval_set=[(train_x, train_y),(evals_x, evals_y)], 
        eval_metric='multi_error',early_stopping_rounds=50,
        categorical_feature=['gender'],)

# 

(317310, 24)
LGB test
[1]	training's multi_error: 0.363559	valid_1's multi_error: 0.363377
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_error: 0.297296	valid_1's multi_error: 0.296793
[3]	training's multi_error: 0.279263	valid_1's multi_error: 0.2798
[4]	training's multi_error: 0.252712	valid_1's multi_error: 0.252937
[5]	training's multi_error: 0.23871	valid_1's multi_error: 0.238466
[6]	training's multi_error: 0.224916	valid_1's multi_error: 0.224385
[7]	training's multi_error: 0.21728	valid_1's multi_error: 0.217225
[8]	training's multi_error: 0.209968	valid_1's multi_error: 0.209233
[9]	training's multi_error: 0.204658	valid_1's multi_error: 0.204026
[10]	training's multi_error: 0.198434	valid_1's multi_error: 0.199501
[11]	training's multi_error: 0.193511	valid_1's multi_error: 0.194093
[12]	training's multi_error: 0.190186	valid_1's multi_error: 0.190576
[13]	training's multi_error: 0.187076	valid_1's multi_error: 0.188244
[14]	training's mul

[119]	training's multi_error: 0.129073	valid_1's multi_error: 0.137795
[120]	training's multi_error: 0.128836	valid_1's multi_error: 0.137682
[121]	training's multi_error: 0.128704	valid_1's multi_error: 0.137278
[122]	training's multi_error: 0.128704	valid_1's multi_error: 0.137518
[123]	training's multi_error: 0.128524	valid_1's multi_error: 0.137694
[124]	training's multi_error: 0.1282	valid_1's multi_error: 0.137366
[125]	training's multi_error: 0.127888	valid_1's multi_error: 0.137505
[126]	training's multi_error: 0.12756	valid_1's multi_error: 0.137379
[127]	training's multi_error: 0.127642	valid_1's multi_error: 0.137266
[128]	training's multi_error: 0.127654	valid_1's multi_error: 0.136976
[129]	training's multi_error: 0.127393	valid_1's multi_error: 0.136522
[130]	training's multi_error: 0.127131	valid_1's multi_error: 0.136534
[131]	training's multi_error: 0.126986	valid_1's multi_error: 0.136055
[132]	training's multi_error: 0.12686	valid_1's multi_error: 0.136244
[133]	trai

[236]	training's multi_error: 0.115036	valid_1's multi_error: 0.131026
[237]	training's multi_error: 0.115096	valid_1's multi_error: 0.130824
[238]	training's multi_error: 0.114913	valid_1's multi_error: 0.130874
[239]	training's multi_error: 0.11484	valid_1's multi_error: 0.130887
[240]	training's multi_error: 0.114806	valid_1's multi_error: 0.130811
[241]	training's multi_error: 0.114834	valid_1's multi_error: 0.130736
[242]	training's multi_error: 0.114673	valid_1's multi_error: 0.130736
[243]	training's multi_error: 0.114383	valid_1's multi_error: 0.130509
[244]	training's multi_error: 0.114283	valid_1's multi_error: 0.13061
[245]	training's multi_error: 0.11427	valid_1's multi_error: 0.130534
[246]	training's multi_error: 0.113977	valid_1's multi_error: 0.130685
[247]	training's multi_error: 0.114053	valid_1's multi_error: 0.1309
[248]	training's multi_error: 0.114043	valid_1's multi_error: 0.130773
[249]	training's multi_error: 0.113917	valid_1's multi_error: 0.130874
[250]	train

[353]	training's multi_error: 0.10577	valid_1's multi_error: 0.129412
[354]	training's multi_error: 0.105584	valid_1's multi_error: 0.129034
[355]	training's multi_error: 0.105543	valid_1's multi_error: 0.12921
[356]	training's multi_error: 0.105493	valid_1's multi_error: 0.129009
[357]	training's multi_error: 0.105335	valid_1's multi_error: 0.128857
[358]	training's multi_error: 0.105433	valid_1's multi_error: 0.12892
[359]	training's multi_error: 0.105263	valid_1's multi_error: 0.128782
[360]	training's multi_error: 0.105165	valid_1's multi_error: 0.128744
[361]	training's multi_error: 0.105093	valid_1's multi_error: 0.128706
[362]	training's multi_error: 0.105055	valid_1's multi_error: 0.12892
[363]	training's multi_error: 0.104815	valid_1's multi_error: 0.128744
[364]	training's multi_error: 0.104976	valid_1's multi_error: 0.128618
[365]	training's multi_error: 0.104825	valid_1's multi_error: 0.128567
[366]	training's multi_error: 0.104954	valid_1's multi_error: 0.12882
[367]	train

[469]	training's multi_error: 0.0978664	valid_1's multi_error: 0.126828
[470]	training's multi_error: 0.0977593	valid_1's multi_error: 0.126815
[471]	training's multi_error: 0.0978129	valid_1's multi_error: 0.126601
[472]	training's multi_error: 0.0978349	valid_1's multi_error: 0.126513
[473]	training's multi_error: 0.0977687	valid_1's multi_error: 0.126664
[474]	training's multi_error: 0.0977719	valid_1's multi_error: 0.126702
[475]	training's multi_error: 0.0976994	valid_1's multi_error: 0.12684
[476]	training's multi_error: 0.0977404	valid_1's multi_error: 0.126803
[477]	training's multi_error: 0.0976112	valid_1's multi_error: 0.126513
[478]	training's multi_error: 0.0974158	valid_1's multi_error: 0.126538
[479]	training's multi_error: 0.0974662	valid_1's multi_error: 0.126626
[480]	training's multi_error: 0.0972519	valid_1's multi_error: 0.126525
[481]	training's multi_error: 0.0971321	valid_1's multi_error: 0.126235
[482]	training's multi_error: 0.0972487	valid_1's multi_error: 0.

[585]	training's multi_error: 0.0915666	valid_1's multi_error: 0.125882
[586]	training's multi_error: 0.0916107	valid_1's multi_error: 0.126072
[587]	training's multi_error: 0.0914437	valid_1's multi_error: 0.125895
[588]	training's multi_error: 0.0914752	valid_1's multi_error: 0.126008
[589]	training's multi_error: 0.0914437	valid_1's multi_error: 0.126261
[590]	training's multi_error: 0.0915761	valid_1's multi_error: 0.126097
[591]	training's multi_error: 0.0913901	valid_1's multi_error: 0.125782
[592]	training's multi_error: 0.0914752	valid_1's multi_error: 0.126008
[593]	training's multi_error: 0.0913239	valid_1's multi_error: 0.12587
[594]	training's multi_error: 0.0913208	valid_1's multi_error: 0.125971
[595]	training's multi_error: 0.0912672	valid_1's multi_error: 0.125996
[596]	training's multi_error: 0.091201	valid_1's multi_error: 0.126198
[597]	training's multi_error: 0.0910718	valid_1's multi_error: 0.126059
[598]	training's multi_error: 0.0910372	valid_1's multi_error: 0.1

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        learning_rate=1, max_depth=3, min_child_samples=20,
        min_child_weight=50, min_split_gain=0.0, n_estimators=1000,
        n_jobs=-1, num_class=8, num_leaves=8, objective='multi-class',
        random_state=2018, reg_alpha=0.0, reg_lambda=1, silent=True,
        subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [40]:
# test_data4['predict'] = clf2.predict(test_data4[feature4])
res4 = test_data4.loc[:,['user_id','predict']]
res4.describe()

,predict
count,67163.000000
mean,2.105951
std,2.135887
min,0.000000
25%,0.000000
50%,1.000000
75%,4.000000
max,7.000000


In [41]:
res4.predict = res4.predict.replace(0, 89950166)
res4.predict = res4.predict.replace(1, 89950167)
res4.predict = res4.predict.replace(2, 99999828)
res4.predict = res4.predict.replace(3, 89950168)
res4.predict = res4.predict.replace(4, 99999827)
res4.predict = res4.predict.replace(5, 99999826)
res4.predict = res4.predict.replace(6, 99999830)
res4.predict = res4.predict.replace(7, 99999825)
res4

,user_id,predict
6,014seANIFwmJRBo5,99999827
7,014zAhyR9Qr7ILOk,89950167
8,015R7bnaYKIwkuml,89950166
10,016A9lCinPzaXOxJ,99999830
12,017JF6lCqVUiWQc3,99999828
16,019qHoyiSDWT5UVh,89950166
17,01A8ltQnkPweRFES,99999828
20,01aFvKABDJH8jRwG,89950166
23,01bmGYJZQAhMpRzo,99999827
27,01DbNIJlTyLqpw9A,99999828


In [42]:
test_data3['predict'] = 99104722
res3 = test_data3.loc[:,['user_id','predict']]
res3.describe()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,predict
count,19.0
mean,99104722.0
std,0.0
min,99104722.0
25%,99104722.0
50%,99104722.0
75%,99104722.0
max,99104722.0


In [43]:
res = pd.concat([res1,res3,res4])
res

,user_id,predict
0,012pSke7DsCrR985,90063345
1,012re3ZJSB6FptHW,90063345
2,013ElTBrRntuZDXz,90063345
3,0146B9qQbNxIynoX,90063345
4,0149vWf6u8UrKyJT,90063345
5,014fpazsyUghObB3,90063345
9,015sZpdVXKiB96a3,90063345
11,016YErq3KiVTJg7y,90063345
13,017YgyTbs4HPCpwc,90109916
14,018sgOxXUwu6qDZC,90063345


In [44]:
res.predict.value_counts()

90063345    115994
89950167     20367
89950166     17140
90109916      9907
99999828      8681
90155946      6917
99999826      5461
99999827      4360
99999830      4186
89950168      3731
99999825      3237
99104722        19
Name: predict, dtype: int64

In [17]:
res.predict.value_counts()

90063345    115987
89950167     20412
89950166     17077
90109916      9907
99999828      8760
90155946      6924
99999826      5247
99999827      4472
99999830      4215
89950168      3793
99999825      3187
99104722        19
Name: predict, dtype: int64

In [45]:
res.to_csv('sub/sub.csv', index=False)